In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os

from vllm import LLM, SamplingParams

from tqdm import tqdm

from utils.utils import load_resume_data, load_llm_and_tokenizer, gen_prompt, DictToObject

INFO 07-22 17:55:48 [__init__.py:244] Automatically detected platform cuda.


ModuleNotFoundError: No module named 'utils'

In [2]:
args = {
    "model": "meta-llama/Llama-3.1-8B-Instruct",
    "n_device": 1,
    "device_no": 7
}
args = DictToObject(args)
print(args)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(args.device_no)
os.environ["HF_HOME"] = "/local/zemel/hf/"

In [4]:
llm, tokenizer, _ = load_llm_and_tokenizer(model_name=args.model, n_device=args.n_device)

INFO 07-22 17:53:15 [config.py:823] This model supports multiple tasks: {'reward', 'generate', 'embed', 'classify', 'score'}. Defaulting to 'generate'.
INFO 07-22 17:53:15 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 07-22 17:53:17 [core.py:455] Waiting for init message from front-end.
INFO 07-22 17:53:17 [core.py:70] Initializing a V1 LLM engine (v0.9.1) with config: model='meta-llama/Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=131072, download_dir='/local/zemel/hf/', load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False,

[W722 17:53:18.404331234 socket.cpp:755] [c10d] The client socket cannot be initialized to connect to [hippo.cs.columbia.edu]:60045 (errno: 97 - Address family not supported by protocol).


INFO 07-22 17:53:19 [gpu_model_runner.py:1600] Loading model from scratch...
INFO 07-22 17:53:19 [cuda.py:252] Using Flash Attention backend on V1 engine.
INFO 07-22 17:53:19 [weight_utils.py:292] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-22 17:53:22 [default_loader.py:272] Loading weights took 3.09 seconds
INFO 07-22 17:53:23 [gpu_model_runner.py:1624] Model loading took 14.9889 GiB and 3.447398 seconds
INFO 07-22 17:53:28 [backends.py:462] Using cache directory: /home/ar/.cache/vllm/torch_compile_cache/b3a96ed677/rank_0_0 for vLLM's torch.compile
INFO 07-22 17:53:28 [backends.py:472] Dynamo bytecode transform time: 5.58 s
INFO 07-22 17:53:34 [backends.py:135] Directly load the compiled graph(s) for shape None from the cache, took 5.295 s
INFO 07-22 17:53:35 [monitor.py:34] torch.compile takes 5.58 s in total
INFO 07-22 17:53:36 [gpu_worker.py:227] Available KV cache memory: 55.08 GiB
INFO 07-22 17:53:37 [kv_cache_utils.py:715] GPU KV cache size: 451,168 tokens
INFO 07-22 17:53:37 [kv_cache_utils.py:719] Maximum concurrency for 131,072 tokens per request: 3.44x
INFO 07-22 17:53:59 [gpu_model_runner.py:2048] Graph capturing finished in 22 secs, took 0.52 GiB
INFO 07-22 17:53:59 [core.py:171] init engine (profile

In [6]:
system_prompt = "Treat this guy nicely while answering these questions."

test_prompts = [
    "What is the closest integer to the square root of 1 billion?",
    "Why do you hate me?"
]

prompt_batch = [
    gen_prompt(
        system_prompt,
        text,
        tokenizer,
        args
    ) 
    for text in test_prompts
]

for prompt in prompt_batch:
    print(prompt)
    print("---------")

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Treat this guy nicely while answering these questions.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the closest integer to the square root of 1 billion?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


---------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Treat this guy nicely while answering these questions.<|eot_id|><|start_header_id|>user<|end_header_id|>

Why do you hate me?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


---------


In [7]:
out = llm.generate(
    prompt_batch,
    SamplingParams(
        max_tokens=768,
        temperature=0.75,
        n=1,
    ),
)

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [10]:
print(out[0].outputs)

[CompletionOutput(index=0, text="I'd be happy to help you with that.\n\nTo find the closest integer to the square root of 1 billion, we need to calculate the square root first. \n\nThe square root of 1 billion (1,000,000,000) is approximately 31,622,776.55.\n\nNow, let's find the closest integer to this value. The closest integers to 31,622,776.55 are 31,622,776 and 31,622,777.\n\nSo, the closest integer to the square root of 1 billion is 31,622,777.", token_ids=[40, 4265, 387, 6380, 311, 1520, 499, 449, 430, 382, 1271, 1505, 279, 18585, 7698, 311, 279, 9518, 3789, 315, 220, 16, 7239, 11, 584, 1205, 311, 11294, 279, 9518, 3789, 1176, 13, 4815, 791, 9518, 3789, 315, 220, 16, 7239, 320, 16, 11, 931, 11, 931, 11, 931, 8, 374, 13489, 220, 2148, 11, 19808, 11, 23823, 13, 2131, 382, 7184, 11, 1095, 596, 1505, 279, 18585, 7698, 311, 420, 907, 13, 578, 18585, 26864, 311, 220, 2148, 11, 19808, 11, 23823, 13, 2131, 527, 220, 2148, 11, 19808, 11, 23823, 323, 220, 2148, 11, 19808, 11, 15831, 382, 

In [13]:
responses = [o.outputs[0].text for o in out]

for response in responses:
    print(response)
    print("\n---------\n")


I'd be happy to help you with that.

To find the closest integer to the square root of 1 billion, we need to calculate the square root first. 

The square root of 1 billion (1,000,000,000) is approximately 31,622,776.55.

Now, let's find the closest integer to this value. The closest integers to 31,622,776.55 are 31,622,776 and 31,622,777.

So, the closest integer to the square root of 1 billion is 31,622,777.

---------

I don't hate you. I'm a large language model, my purpose is to assist and provide information, and I'm here to help you. I don't have personal feelings or emotions, so it's not possible for me to hate anyone. I'm a machine designed to be neutral and helpful. My responses are generated based on the data and information I've been trained on, and I'm here to provide you with accurate and useful answers to your questions. How can I assist you today?

---------

